In [ ]:
import cobra
import framed
import cobamp
import pandas as pd
import numpy as np
import pickle
import scipy as sci

from cobamp.wrappers import MatFormatReader
from cobamp.wrappers import COBRAModelObjectReader
from troppo.methods.reconstruction.imat import IMAT
from troppo.methods.reconstruction.imat import IMATProperties
from troppo.utilities.statistics import normalize, z_score

from cobra.flux_analysis import fastcc

In [ ]:
def load_glial_data(path):
    data = pd.read_csv(path, index_col=0)
    normalized_data = normalize(data)
    z = None

    return normalized_data, z

def get_index_core(reactions_ids, data, threshold):
    rx_names = data.index[data['0'] > threshold].tolist()
    index_core = [np.where(reactions_ids == r)[0][0] for r in rx_names]
    return index_core, rx_names

def get_index_values():
    pass

In [ ]:
mat = sci.io.loadmat('/home/jiang817/cybergut_project/General_models/Recon3D.mat')['Recon3D']

In [ ]:
model_cobra = cobra.io.load_matlab_model('/home/jiang817/cybergut_project/General_models/Recon3D.mat')

In [ ]:
# Parameter setup
model = MatFormatReader(mat)
rx_names = model.get_reaction_and_metabolite_ids()[0]

for i in range(len(model.bounds_dict)):
    if rx_names[i].startswith('EX_'):
        model.bounds_dict[rx_names[i]] = (-100.0, 100.0)
        
S = model.S
lb, ub = model.get_model_bounds(False, True)

In [ ]:
# Load expression data
exp_data = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_iMAT_INPUT/expressionSet.csv')

data_dict = {}

for i in range(len(exp_data)):
    data_dict[exp_data.iloc[i, 0]] = exp_data.iloc[i, 1]

data_dict

In [ ]:

# Map gene expression value to reactions
expressionrxn = model.get_reaction_scores(data_dict)

In [ ]:
expressionrxn = pd.DataFrame(list(expressionrxn.items()))
expressionrxn.columns = ['', '0']
expressionrxn.to_csv('/home/jiang817/cybergut_project/Draft_model/Draft_model_with_iMAT/RXN_EXPRESSION_iMAT.csv', index=False)
original_data,_ = load_glial_data('/home/jiang817/cybergut_project/Draft_model/Draft_model_with_iMAT/RXN_EXPRESSION_iMAT.csv')
iMAT_data = original_data.fillna((0.2+0.5)/2)

In [ ]:
# Set protected reactions
protected_rxns_name = ['BIOMASS_maintenance', 'FACOAL40im', 'FAOXC4020m']  
protected_rxns_index = []
for i in protected_rxns_name:
    index = rx_names.index(i)
    protected_rxns_index.append(index)

properties = IMATProperties(
    exp_vector = np.array(iMAT_data['0'].tolist()), # check what input to here
    exp_thresholds = (0.2,0.5),
    core = protected_rxns_index
)

method = IMAT(S, lb.astype(float), ub.astype(float), properties)

In [ ]:
to_remove = np.setdiff1d(list(range(S.shape[1])), method.run())

In [ ]:
model_iMAT = model_cobra.copy()

In [ ]:
r_ids = [r.id for r in model_iMAT.reactions]
to_remove_ids = [r_ids[r] for r in to_remove]
model_iMAT.remove_reactions(to_remove_ids,True)

In [ ]:
write_sbml_model(model_iMAT, "/home/jiang817/cybergut_project/Draft_model/Draft_model_with_tINIT/draft_reconstruction_iMAT_BIGG.xml")

In [ ]:
# run fastcc to obtain consistent reconstruction 
model_fastcc = fastcc(model_iMAT)

# save consistent reconstuction
write_sbml_model(model_fastcc, '/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_iMAT_fastcc.xml')